In [1]:
import pandas as pd
import numpy as np
import time
import math
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from ete3 import NCBITaxa
ncbi = NCBITaxa()

## Commands
### CDS
cat blastp_nr_karect26_300_CDS.txt | grep "Query= \|^gi|" > blastp_nr_CDS

cat blastp_nr_CDS | sed -e 's/|  /,,,/g'| sed -e 's/         /,,,/g'|sed -e 's/        /,,,/g' |sed -e 's/       /,,,/g' | sed -e 's/      /,,,/g' |sed -e 's/     /,,,/g' | sed -e 's/    /,,,/g' | sed -e 's/   /,,,/g'| sed -e 's/  /,,,/g'| sed -e 's/|,/,,,/g'| sed -e 's/gi|//g'| sed -e 's/|gb|/,,,gb,,,/g'| sed -e 's/|ref|/,,,ref,,,/g'| sed -e 's/|emb|/,,,emb,,,/g'| sed -e 's/|pdb|/,,,pdb,,,/g' | sed -e 's/|dbj|/,,,dbj,,,/g' | sed -e 's/|tpg|/,,,tpg,,,/g'| sed -e 's/|sp|/,,,sp,,,/g'| sed -e 's/|prf|/,,,prf,,,/g'| sed -e 's/|pir|/,,,pir,,,/g'| sed -e 's/|tpe|/,,,tpe,,,/g' > temp_CDS

### contigs
cat blastp_nr_karect26_300_contigs.txt | grep "Query= \|^gi|" > blastp_nr_contigs

cat blastp_nr_contigs | sed -e 's/|  /,,,/g'| sed -e 's/         /,,,/g'|sed -e 's/        /,,,/g' |sed -e 's/       /,,,/g' | sed -e 's/      /,,,/g' |sed -e 's/     /,,,/g' | sed -e 's/    /,,,/g' | sed -e 's/   /,,,/g'| sed -e 's/  /,,,/g'| sed -e 's/|,/,,,/g'| sed -e 's/gi|//g'| sed -e 's/|gb|/,,,gb,,,/g'| sed -e 's/|ref|/,,,ref,,,/g'| sed -e 's/|emb|/,,,emb,,,/g'| sed -e 's/|pdb|/,,,pdb,,,/g' | sed -e 's/|dbj|/,,,dbj,,,/g' | sed -e 's/|tpg|/,,,tpg,,,/g'| sed -e 's/|sp|/,,,sp,,,/g'| sed -e 's/|prf|/,,,prf,,,/g'| sed -e 's/|pir|/,,,pir,,,/g'| sed -e 's/|tpe|/,,,tpe,,,/g' > temp_contigs

## Input BLAST results

In [2]:
col = [0,1,2,3,4,5,6,7,8,9,10]

tmp_CDS = pd.read_csv('Data/temp_CDS', header=None, names=col, sep=',,,', engine='python')
tmp_contigs = pd.read_csv('Data/temp_contigs', header=None, names=col, sep=',,,', engine='python')

## Rearrangement

In [3]:
def Del_NA(df, startcol, endcol):
    
    for c in range(startcol, endcol):
        
        index_list = list(df.dropna(subset=[c])[df.dropna(subset=[c])[c+1].isna()].index)
        
        for i in range(len(index_list)):
            df.iat[index_list[i], 4] = df.iat[index_list[i], c-1]
            df.iat[index_list[i], 5] = df.iat[index_list[i], c]
    
    dfout =df[[0,2,4,5]]
    dfout.columns = ['gi','accession','score','e-value']

    return dfout

def Rearrangement_BLASTresult(df):
    dfdelna = Del_NA(df, 6, 10)
    
    # query identifier list 
    seqlist = dfdelna[dfdelna['gi'].str.contains('Query=')]['gi'].str.replace('Query= ', '').str.split(' ', expand=True).loc[:,[0]]
    dfout = pd.concat([dfdelna, seqlist[0]], axis=1, join_axes=[dfdelna.index])

    # fill query identifier
    dfout[0] = dfout[0].fillna(method="ffill")
    dfout = dfout.rename(columns={0:'SeqIden'})
    dfout['TrinityIden'] = dfout['SeqIden'].str.replace('|','\t').str.split('\t', expand=True)[0]

    # delete col (Query=)
    dfout = dfout.drop(seqlist.index)
    dfout['rank'] = range(len(dfout))
    
    # all sequence list
    seqlist[1] = seqlist[0].str.replace('|','\t').str.split('\t', expand=True)[0]
    seqlist.columns=['SeqIden','TrinityIden']

    return seqlist, dfout

In [4]:
CDSSeqlist, CDSNR= Rearrangement_BLASTresult(tmp_CDS)
CDSNR.head()

ContigSeqlist, ContigNR= Rearrangement_BLASTresult(tmp_contigs)
ContigNR.head()

,gi,accession,score,e-value,SeqIden,TrinityIden,rank
1,1088597648,OHE15404.1,205,2.000000e-61,TRINITY_DN10013_c0_g1_i1,TRINITY_DN10013_c0_g1_i1,0
2,1088600340,OHE17577.1,204,6.000000e-61,TRINITY_DN10013_c0_g1_i1,TRINITY_DN10013_c0_g1_i1,1
3,1334725090,PNV84571.1,208,8.000000e-61,TRINITY_DN10013_c0_g1_i1,TRINITY_DN10013_c0_g1_i1,2
4,1088600246,OHE17493.1,204,1.000000e-60,TRINITY_DN10013_c0_g1_i1,TRINITY_DN10013_c0_g1_i1,3
5,1243954385,WP_096047732.1,207,2.000000e-60,TRINITY_DN10013_c0_g1_i1,TRINITY_DN10013_c0_g1_i1,4


In [5]:
print("#All Contigs:", len(set(ContigSeqlist['SeqIden'])))
print("#All CDSs:", len(set(CDSSeqlist['SeqIden'])))
print("#Anotated CDSs:", len(set(CDSNR['SeqIden'])))
print("#Anotated CDS (Contig):", len(set(CDSNR['TrinityIden'])))
print("#Anotated Contigs:", len(set(ContigNR['SeqIden']))) #*
print("#Anotated Contigs:", len(set(ContigNR['TrinityIden'])))

#All Contigs: 146985
#All CDSs: 109253
#Anotated CDSs: 97944
#Anotated CDS (Contig): 87029
#Anotated Contigs: 126896
#Anotated Contigs: 126896


In [6]:
# CDS without annotation
CDSNoNR=CDSSeqlist[~CDSSeqlist['SeqIden'].isin(set(CDSNR['SeqIden']))]
print(len(set(CDSNoNR['SeqIden'])))

# contigs without CDS
NoCDSContig=ContigSeqlist[~ContigSeqlist['TrinityIden'].isin(set(CDSSeqlist['TrinityIden']))]
print(len(set(NoCDSContig['TrinityIden'])))

# NoCDSContig without annotation
NoCDSContigNoNR=NoCDSContig[~NoCDSContig['TrinityIden'].isin(set(ContigNR['TrinityIden']))]
print(len(set(NoCDSContigNoNR['TrinityIden'])))

# NoCDSContig with annotation
tmp=NoCDSContig[~NoCDSContig['TrinityIden'].isin(set(NoCDSContigNoNR['TrinityIden']))]

NoCDSContigNR=ContigNR[ContigNR['TrinityIden'].isin(set(tmp['TrinityIden']))]
print(len(set(NoCDSContigNR['TrinityIden'])))

11309
49637
10634
39003


In [15]:
df = pd.concat([CDSNR, NoCDSContigNR])
print(len(set(df['SeqIden'])))
print(len(set(df['TrinityIden'])))

136947
126032


## Retrieve top hits assigned to major taxa

In [9]:
pro_list1 = pd.read_table('/nfs_share/motoki/metatra/201809_metatra/data/ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid', usecols=[1,2], header=0, names=('accession','taxid'))
pro_list2 = pd.read_table('/nfs_share/motoki/metatra/201809_metatra/data/ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/pdb.accession2taxid', usecols=[1,2], header=0, names=('accession','taxid'))
acc2taxid = pd.concat([pro_list1, pro_list2], sort=False)

In [16]:
def dominant_blast_results():
    
    # 1 major taxa's taxid
    MajorTaxid = []
    
    for i in range(len(taxalist)):
        MajorTaxid.extend(ncbi.get_descendant_taxa(taxalist[i]))
        
    MajorTaxid = set(MajorTaxid)

    # 2 major taxa's accession
    MajorAcc = set(acc2taxid[acc2taxid['taxid'].isin(MajorTaxid)]['accession'])
    print('MajorAccession: ', len(MajorAcc)) 

    # 3 extaract data from blast result
    dfmajor = df[df['accession'].isin(MajorAcc)]
    dfnotmajor = df[~df['SeqIden'].isin(set(dfmajor['SeqIden']))]
    
    return dfmajor, dfnotmajor

In [17]:
taxalist = ['Sulfurovum', 'Methylococcales', 'Thiotrichales']
dfmajor, dfnotmajor = dominant_blast_results()

print(len(set(dfmajor['TrinityIden'])))
print(len(set(dfmajor['SeqIden'])))

print(len(set(dfnotmajor['TrinityIden'])))
print(len(set(dfnotmajor['SeqIden'])))

MajorAccession:  834903
91926
100579
35313
36368


In [12]:
def Retrieve_tophits(dfdat):
    
    SeqIden = list(set(dfdat["SeqIden"]))
    dfoutput = pd.DataFrame(columns=dfdat.columns)

    for i in range(len(SeqIden)):
        
        # largest bitscore
        tmp = dfdat[(dfdat["SeqIden"] == SeqIden[i])]
        bitMAX = tmp[(tmp.score.astype(float) == tmp.score.astype(float).max())]
                
        if len(bitMAX) == 1:
            dfoutput.loc[i]=bitMAX.iloc[0,]
            
        # bitscore in a tie: minimum evalue
        elif len(bitMAX) != 1:
            EvalueMIN = bitMAX[(bitMAX['e-value'].astype(float) == bitMAX['e-value'].astype(float).min())]
            
            if len(EvalueMIN) == 1:
                dfoutput.loc[i]=EvalueMIN.iloc[0,]
                
            # both of bitscore and evalue in a tie: first row
            else:
                dfoutput.loc[i]=EvalueMIN.iloc[0,]
            
    return dfoutput


def Divid_Seq(dfdat):
    
    dfoutput = pd.DataFrame(columns=list(dfdat.columns))
    
    SeqIden = list(set(dfdat["SeqIden"]))

    sep = 5000
    sepfiles = round(len(SeqIden)/sep) + 1
    
    for i in range(sepfiles):
        
        # extract 'sep' of each sequence
        Sequences = set(SeqIden[sep*i:sep*(i+1)])
        tmp = dfdat[dfdat["SeqIden"].isin(Sequences)]
        
        output = Retrieve_tophits(tmp)
        
        # confirm < 5000 ('sep')
        if i == sepfiles -1:
            print('sequences:', len(list(set(tmp["SeqIden"]))))
            print(sepfiles)
    
        dfoutput = pd.concat([dfoutput, output])
        
    return dfoutput

In [18]:
dfmajor_tophit = Divid_Seq(dfmajor)

sequences: 579
21


In [19]:
dfnotmajor_tophit = Divid_Seq(dfnotmajor)

sequences: 1368
8


In [20]:
dftophit = pd.concat([dfmajor_tophit, dfnotmajor_tophit])
print(len(dftophit))

136947


# Get taxonomic information for accession number

In [21]:
AssignedAcc = acc2taxid[acc2taxid['accession'].isin(set(dftophit['accession']))]
taxid_list = list(set(AssignedAcc['taxid']))

In [22]:
def get_swap_dict(d):
    return {v: k for k, v in d.items()}


def get_all_taxid(taxid_list):
    
    df = pd.DataFrame(columns=['superkingdom','phylum','class','subphylum','family','genus','order','species'])

    for i in range(len(taxid_list)):
        lineage = ncbi.get_lineage(taxid_list[i])
        data = ncbi.get_rank(lineage)

        d_swap = get_swap_dict(data)

        df = pd.concat([df, pd.DataFrame(d_swap, index=[taxid_list[i],])], sort=False)

    df = df.fillna(0)  # No taxid 0 in NCBItaxa
    return df


def get_all_taxid_info(taxid_list):

    dfoutput = get_all_taxid(taxid_list)
    
    for j in range(len(taxid_list)):
        the_list = list(dfoutput.loc[taxid_list[j],])

        for i in range(len(the_list)):
            if the_list[i] == 0:
                continue

            info = ncbi.get_taxid_translator([the_list[i]])

            dfoutput.iloc[j:j+1,i] = info[the_list[i]]
            
    dfoutput = dfoutput.reset_index()
    dfoutput = dfoutput.rename(columns={'index':'taxid'})
            
    return dfoutput

In [23]:
TaxidInfo = get_all_taxid_info(taxid_list)

dfinfo = pd.merge(AssignedAcc, TaxidInfo.loc[:,'taxid':'species'], on='taxid', how='left')
dftophit_taxa = pd.merge(dftophit, dfinfo, on='accession', how='left')

In [24]:
dfNR = pd.concat([dftophit_taxa, CDSNoNR, NoCDSContigNoNR], ignore_index=True, sort=False)

In [25]:
print("all data", len(dfNR))  # 156430
print("all data (CDSs)", len(set(dfNR["SeqIden"])))  # 146985
print("all data (contigs)", len(set(dfNR["TrinityIden"])))  # 146985

all data 158890
all data (CDSs) 158890
all data (contigs) 146985


## Input KEGG annotation and TPM

In [26]:
# KEGG annotation
KOCDS = pd.read_csv('Data/out_out_out_blastp_tophit_nr_S', skiprows=[0], header=None, sep='\t',
                   usecols=[0,1,4,23], names=['SeqIden','Qlen','Slen','KO'])

KOContigs = pd.read_csv('Data/out_out_out_blastx_tophit_nr_S', skiprows=[0], header=None, sep='\t',
                   usecols=[0,1,4,23], names=['SeqIden','Qlen','Slen','KO'])

KOs = pd.concat([KOCDS, KOContigs])


# TPM
insitu1 = pd.read_csv(
    'Data/insitu_4_1_2.isoforms.results', 
    header=None, skiprows=[0], sep='\t', usecols=[0,2,3,4,5], 
    names=['TrinityIden','gene_id_len','expected_len','insitu1_count','insitu1']
)

insitu2 = pd.read_csv(
    'Data/insitu_8_3.isoforms.results', 
    header=None, skiprows=[0], sep='\t', usecols=[0,4,5], names=['TrinityIden','insitu2_count','insitu2']
)

onboard1 = pd.read_csv(
    'Data/onboard_2_1_2.isoforms.results', 
    header=None, skiprows=[0], sep='\t', usecols=[0,4,5], names=['TrinityIden','onboard1_count','onboard1']
)

onboard2 = pd.read_csv(
    'Data/onboard_10_1_2.isoforms.results', 
    header=None, skiprows=[0], sep='\t', usecols=[0,4,5], names=['TrinityIden','onboard2_count','onboard2']
)

dfTPM = pd.merge(insitu1, insitu2, on='TrinityIden')
dfTPM = pd.merge(dfTPM, onboard1, on='TrinityIden')
dfTPM = pd.merge(dfTPM, onboard2, on='TrinityIden')

print(len(dfTPM))   # 146985


146985


In [27]:
dfNRKG = pd.merge(dfNR, KOs, on='SeqIden', how='left')
dfNRKG_tpm = pd.merge(dfNRKG, dfTPM, on='TrinityIden', how='left')

print(len(dfNRKG_tpm))  # 158890
print(len(set(dfNRKG_tpm['TrinityIden'])))   # 146985

158890
146985


## Output

In [28]:
dfNRKG_tpm.to_csv("Data/NRKG_tpm", sep="\t")